In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv('/content/drive/Shared drives/Sarcasm Detection/Malayalam/sarcasm_mal_train (1).csv',usecols=['Text','labels'])
df['labels']=df['labels'].replace({'Non-sarcastic':0,'Sarcastic':1})
df.dropna()
df

,Text,labels
0,Screenshot edukkan vannth njan മാത്രമാണോ,1
1,നമ്മുടെ അനു സിത്താര ചേച്ചി ഇങ്ങനെ വരുന്നത് നോക...,1
2,Mollyhood is getting bigger and bigger,0
3,Ho aaa BGM. Mammookka ithu oru pwoli pwolikkum,0
4,"Enthaale, sambhavam puraanam aanelum backgroun...",1
...,...,...
13183,Madhu C Narayanan .... പ്രതീക്ഷ ഉള്ള ഒരു സംവിധ...,0
13184,🤣🤣🤣🤣🤣 kya mazak hai... Hans hans k lotpot ho j...,0
13185,channel aaanu ishtapettal subscribe cheyyumo plz,0
13186,Nte ponno... kidilam... marana waiting,0


# Pre Processing

Stopwords Removal

In [ ]:
X=df['Text']
y=df['labels']

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    text = re.sub(r'\d+', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

def remove_numeric_values(text):
    return re.sub(r'\d+', '', text)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=32)

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

TF-IDF Vectorizer


In [ ]:
# Step 1: Install necessary libraries
!pip install scikit-learn

# Step 2: Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

df['Preprocessed_Text'] = df['Text'].apply(preprocess)
df['Preprocessed_Text'] = df['Text'].apply(remove_numeric_values)

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5, token_pattern=r'\b\w+\b')

# Apply TF-IDF vectorization on preprocessed text
X = vectorizer.fit_transform(df['Preprocessed_Text'])
y = df['labels']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Upsampling

In [ ]:
print(y_train.value_counts())

labels
0    8547
1    2003
Name: count, dtype: int64


In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE(random_state = 42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print(y_train_res.value_counts())

labels
0    8547
1    8547
Name: count, dtype: int64


# Models

SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:
svm_model = SVC(kernel='linear', random_state=42)

svm_model.fit(X_train_res,y_train_res)

SVC(kernel='linear', random_state=42)

In [ ]:
y_svm = svm_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_svm))

              precision    recall  f1-score   support

           0       0.91      0.77      0.83      2142
           1       0.40      0.66      0.50       496

    accuracy                           0.75      2638
   macro avg       0.65      0.72      0.67      2638
weighted avg       0.81      0.75      0.77      2638



Logisitc Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr1 = LogisticRegression()
lr1.fit(X_train_res, y_train_res.ravel())
predictions = lr1.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.91      0.79      0.85      2142
           1       0.42      0.65      0.51       496

    accuracy                           0.77      2638
   macro avg       0.67      0.72      0.68      2638
weighted avg       0.82      0.77      0.78      2638



KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train_res,y_train_res)
predictions_knn= knn.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions_knn))

              precision    recall  f1-score   support

           0       0.92      0.21      0.34      2142
           1       0.21      0.92      0.34       496

    accuracy                           0.34      2638
   macro avg       0.56      0.56      0.34      2638
weighted avg       0.78      0.34      0.34      2638



Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train_res,y_train_res)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88      2142
           1       0.45      0.40      0.43       496

    accuracy                           0.80      2638
   macro avg       0.66      0.65      0.65      2638
weighted avg       0.79      0.80      0.79      2638



Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)

dt_model.fit(X_train_res,y_train_res)

DecisionTreeClassifier(random_state=42)

In [ ]:
y_dectree = dt_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_dectree))

              precision    recall  f1-score   support

           0       0.85      0.82      0.83      2142
           1       0.33      0.38      0.35       496

    accuracy                           0.74      2638
   macro avg       0.59      0.60      0.59      2638
weighted avg       0.75      0.74      0.74      2638



Ensemble - All 5

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
svm_clf = SVC(kernel='linear', C=1.0, probability=True)
dt_clf = DecisionTreeClassifier(random_state=42)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
log_reg_clf = LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
ensemble_clf = VotingClassifier(estimators=[
    ('knn', knn_clf),
    ('svm', svm_clf),
    ('dt', dt_clf),
    ('rf', rf_clf),
    ('log_reg', log_reg_clf)
], voting='soft')

In [ ]:
ensemble_clf.fit(X_train_res,y_train_res)

VotingClassifier(estimators=[('knn', KNeighborsClassifier()),
                             ('svm', SVC(kernel='linear', probability=True)),
                             ('dt', DecisionTreeClassifier(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('log_reg',
                              LogisticRegression(max_iter=1000,
                                                 random_state=42))],
                 voting='soft')

In [ ]:
ensemble_pred = ensemble_clf.predict(X_test)

In [ ]:
print(classification_report(y_test, ensemble_pred))

              precision    recall  f1-score   support

           0       0.90      0.77      0.83      2142
           1       0.40      0.64      0.49       496

    accuracy                           0.75      2638
   macro avg       0.65      0.71      0.66      2638
weighted avg       0.81      0.75      0.77      2638



# Result


In [ ]:
df2 = pd.read_csv('/content/drive/Shared drives/Sarcasm Detection/Malayalam/sarcasm_mal_test_without_labels.csv')
df2

,ID,Text
0,Id_01,Shavakallarayile Kuzhimaadathile Peril Oru Let...
1,Id_02,ഗീതു മോഹൻദാസ് മലയാള സിനിമക്കു നൽകുന്ന വമ്പൻ ഗി...
2,Id_03,Ente ponno ah sound🥰🥰 poli poli🤘
3,Id_04,Villain sharafudheen ennu thonnunnavar likikk...
4,Id_05,pulimurukan trailer ano kanunath 🤔
...,...,...
2821,Id_2822,Ente ponno oru adaaru jagapoka aanenu manasila...
2822,Id_2823,എന്റെ ഇക്ക nja നമിച്ചു... ഒരു രക്ഷയില്ല ഹെവി ഐ...
2823,Id_2824,ദേ ഇപ്പൊ കണ്ട് ഇറങ്ങിയതേ ഉള്ളു 96 Karikku (+...
2824,Id_2825,1) Drisyam 2) Memories 3) Seconds 4) Grand ma...


In [ ]:
X=df2['Text']
X=X.apply(preprocess)
X = vectorizer.transform(X)
print(X.shape)

(2826, 2847)


SVM Prediction

In [ ]:
svm_test_pred=svm_model.predict(X)

Logisitic Regression Prediction

In [ ]:
lr_test_pred= lr1.predict(X)
print(lr_test_pred.shape)

(2826,)


KNN Prediction

In [ ]:
knn_test_pred= knn.predict(X)

Random Forest Prediction

In [ ]:
print(X.shape)
rf_test_pred=rf_model.predict(X)

(2826, 2847)


Decision Tree Prediction

In [ ]:
dt_test_pred= dt_model.predict(X)

Ensemble Prediction

In [ ]:
ensemble_test_pred=ensemble_clf.predict(X)
ensemble_test_pred

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
ensemble_test_pred_df = pd.DataFrame(ensemble_test_pred, columns = ['labels'])
ensemble_test_pred_df['labels']=ensemble_test_pred_df['labels'].replace({0:'Non-sarcastic',1:'Sarcastic'})
ensemble_test_pred_df
df_combined = pd.concat([df2, ensemble_test_pred_df], axis=1)
df_combined

,ID,Text,labels
0,Id_01,Shavakallarayile Kuzhimaadathile Peril Oru Let...,Non-sarcastic
1,Id_02,ഗീതു മോഹൻദാസ് മലയാള സിനിമക്കു നൽകുന്ന വമ്പൻ ഗി...,Non-sarcastic
2,Id_03,Ente ponno ah sound🥰🥰 poli poli🤘,Non-sarcastic
3,Id_04,Villain sharafudheen ennu thonnunnavar likikk...,Non-sarcastic
4,Id_05,pulimurukan trailer ano kanunath 🤔,Non-sarcastic
...,...,...,...
2821,Id_2822,Ente ponno oru adaaru jagapoka aanenu manasila...,Non-sarcastic
2822,Id_2823,എന്റെ ഇക്ക nja നമിച്ചു... ഒരു രക്ഷയില്ല ഹെവി ഐ...,Non-sarcastic
2823,Id_2824,ദേ ഇപ്പൊ കണ്ട് ഇറങ്ങിയതേ ഉള്ളു 96 Karikku (+...,Sarcastic
2824,Id_2825,1) Drisyam 2) Memories 3) Seconds 4) Grand ma...,Non-sarcastic


In [ ]:
df_combined.to_csv('/content/drive/Shared drives/Sarcasm Detection/Final - Malayalam/output_malayalam_final.csv', sep=",", index=False, header=["ID", "Text", "Predicted_Labels"])